# CNN for AdvSND energy reconstruction

In [ ]:
from tensorflow.keras.layers import (
    BatchNormalization,
    Concatenate,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    Input,
    MaxPooling2D,
    ReLU,
)
from tensorflow.keras.models import Model

In [ ]:
from config import input_shape, input_shape_mf

In [ ]:
import tensorflow.keras.backend as K

K.set_image_data_format("channels_last")

In [ ]:
model_name = "CNN_jannu_energy_pi_zero"

In [ ]:
drop_middle = 0.25

lr = 2e-4

# AdvTarget

target_h_input = Input(input_shape, name="target_h_in")
X_h = Conv2D(16, kernel_size=(1, 9), padding="same")(target_h_input)
X_h = BatchNormalization()(X_h)
X_h = ReLU()(X_h)
X_h = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_h)
X_h = Dropout(rate=drop_middle)(X_h)

X_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_h)
X_h = BatchNormalization()(X_h)
X_h = ReLU()(X_h)
X_h = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_h)
X_h = Dropout(rate=drop_middle)(X_h)

X_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_h)
X_h = BatchNormalization()(X_h)
X_h = ReLU()(X_h)
X_h = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_h)
X_h = Dropout(rate=drop_middle)(X_h)

X_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_h)
X_h = BatchNormalization()(X_h)
X_h = ReLU()(X_h)
X_h = MaxPooling2D(pool_size=(2, 2), padding="same")(X_h)

X_h = Flatten()(X_h)

target_v_input = Input(input_shape, name="target_v_in")
X_v = Conv2D(16, kernel_size=(1, 9), padding="same")(target_v_input)
X_v = BatchNormalization()(X_v)
X_v = ReLU()(X_v)
X_v = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_v)
X_v = Dropout(rate=drop_middle)(X_v)

X_v = Conv2D(16, kernel_size=(3, 3), padding="same")(X_v)
X_v = BatchNormalization()(X_v)
X_v = ReLU()(X_v)
X_v = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_v)
X_v = Dropout(rate=drop_middle)(X_v)

X_v = Conv2D(16, kernel_size=(3, 3), padding="same")(X_v)
X_v = BatchNormalization()(X_v)
X_v = ReLU()(X_v)
X_v = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_v)
X_v = Dropout(rate=drop_middle)(X_v)

X_v = Conv2D(16, kernel_size=(3, 3), padding="same")(X_v)
X_v = BatchNormalization()(X_v)
X_v = ReLU()(X_v)
X_v = MaxPooling2D(pool_size=(2, 2), padding="same")(X_v)

X_v = Flatten()(X_v)

# AdvTarget

mufilter_h_input = Input(input_shape_mf, name="mufilter_h_in")
X_mf_h = Conv2D(16, kernel_size=(1, 3), padding="same")(mufilter_h_input)
X_mf_h = BatchNormalization()(X_mf_h)
X_mf_h = ReLU()(X_mf_h)
X_mf_h = MaxPooling2D(pool_size=(1, 4), padding="valid")(X_mf_h)
X_mf_h = Dropout(rate=drop_middle)(X_mf_h)

X_mf_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_mf_h)
X_mf_h = BatchNormalization()(X_mf_h)
X_mf_h = ReLU()(X_mf_h)
X_mf_h = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_mf_h)
X_mf_h = Dropout(rate=drop_middle)(X_mf_h)

X_mf_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_mf_h)
X_mf_h = BatchNormalization()(X_mf_h)
X_mf_h = ReLU()(X_mf_h)
X_mf_h = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_mf_h)
X_mf_h = Dropout(rate=drop_middle)(X_mf_h)

X_mf_h = Conv2D(16, kernel_size=(3, 3), padding="same")(X_mf_h)
X_mf_h = BatchNormalization()(X_mf_h)
X_mf_h = ReLU()(X_mf_h)
X_mf_h = MaxPooling2D(pool_size=(2, 2), padding="same")(X_mf_h)

X_mf_h = Flatten()(X_mf_h)

mufilter_v_input = Input(input_shape_mf, name="mufilter_v_in")
X_mf_v = Conv2D(16, kernel_size=(1, 3), padding="same")(mufilter_v_input)
X_mf_v = BatchNormalization()(X_mf_v)
X_mf_v = ReLU()(X_mf_v)
X_mf_v = MaxPooling2D(pool_size=(1, 4), padding="valid")(X_mf_v)
X_mf_v = Dropout(rate=drop_middle)(X_mf_v)

X_mf_v = Conv2D(16, kernel_size=(3, 3), padding="same")(X_mf_v)
X_mf_v = BatchNormalization()(X_mf_v)
X_mf_v = ReLU()(X_mf_v)
X_mf_v = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_mf_v)
X_mf_v = Dropout(rate=drop_middle)(X_mf_v)

X_mf_v = Conv2D(16, kernel_size=(3, 3), padding="same")(X_mf_v)
X_mf_v = BatchNormalization()(X_mf_v)
X_mf_v = ReLU()(X_mf_v)
X_mf_v = MaxPooling2D(pool_size=(2, 4), padding="valid")(X_mf_v)
X_mf_v = Dropout(rate=drop_middle)(X_mf_v)

X_mf_v = Conv2D(16, kernel_size=(3, 3), padding="same")(X_mf_v)
X_mf_v = BatchNormalization()(X_mf_v)
X_mf_v = ReLU()(X_mf_v)
X_mf_v = MaxPooling2D(pool_size=(2, 2), padding="same")(X_mf_v)

X_mf_v = Flatten()(X_mf_v)

X = Concatenate()([X_h, X_v, X_mf_h, X_mf_v])
X = Dense(4)(X)
X = BatchNormalization()(X)
X = ReLU()(X)
X = Dense(20)(X)
X = BatchNormalization()(X)
X = ReLU()(X)
X = Dropout(rate=0.2)(X)
X = Dense(1)(X)

model = Model(
    inputs=[target_h_input, target_v_input, mufilter_h_input, mufilter_v_input],
    outputs=X,
    name=model_name,
)

K.clear_session()

model.compile(
    optimizer="Adam",
    loss="mse",
    metrics=[
        "mae",
    ],
)

In [ ]:
model.save(f"{model_name}_e0.keras")